In [ ]:
import pandas as pd
train_data = pd.read_csv('https://raw.githubusercontent.com/NHANGIOI/Machine_Learning/main/home-data-for-ml-course/train.csv')
test_data = pd.read_csv('https://raw.githubusercontent.com/NHANGIOI/Machine_Learning/main/home-data-for-ml-course/test.csv')


In [ ]:
# Numeric columns
num_cols = train_data.select_dtypes(include=['int64','float64']).columns.drop('SalePrice')
cat_cols = train_data.select_dtypes(include=['object']).columns

# Fill NaN
train_data[num_cols] = train_data[num_cols].fillna(train_data[num_cols].median())
test_data[num_cols] = test_data[num_cols].fillna(train_data[num_cols].median())

train_data[cat_cols] = train_data[cat_cols].fillna(train_data[cat_cols].mode().iloc[0])
test_data[cat_cols] = test_data[cat_cols].fillna(train_data[cat_cols].mode().iloc[0])

# Convert categorical to dummies
data_features = pd.get_dummies(train_data.drop('SalePrice', axis=1))
test_features = pd.get_dummies(test_data)

# Align test_features columns với train_features
test_features = test_features.reindex(columns=data_features.columns, fill_value=0)

data_target = train_data['SalePrice']

In [ ]:
from sklearn.model_selection import train_test_split
train_features, val_features, train_target, val_target = train_test_split(data_features, data_target,random_state=1)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def run_model_forest(siz_leaf : int, out_csv : bool):
    model = RandomForestRegressor(max_leaf_nodes = siz_leaf,random_state = 1)
    model.fit(train_features,train_target)
    MAE = mean_absolute_error(val_target,model.predict(val_features))
    if(out_csv == True):
        submition = pd.DataFrame({
            'Id': test_data['Id'],
            'SalePrice': model.predict(test_features)
        })
        return submition
    else:
        return MAE

def run_model(siz_leaf : int,out_csv : bool):
    model = DecisionTreeRegressor(max_leaf_nodes = siz_leaf,random_state=1)
    model.fit(train_features,train_target)
    MAE = mean_absolute_error(val_target,model.predict(val_features))
    if(out_csv == True):
        submition = pd.DataFrame({
            'Id': test_data['Id'],
            'SalePrice': model.predict(test_features)
        })
        return submition
    else:
        return MAE

In [ ]:
opt_siz = 2
minn_mae = run_model(opt_siz,False)
for i in range(3,100):
    res = run_model(i,False)
    if(minn_mae > res):
        opt_siz = i
        minn_mae = res
        print(i, ": BETTER")
file_submit = run_model(opt_siz,True).to_csv('submit.csv', index=False)
#file_submit.download("/content/submit.csv")